# Partie 1 : Préparation et normalisation des jeux de données

Dans cette première partie, nous préparons et normalisons les quatre jeux de données utilisés dans l’étude du CROUS de l’Île-de-France.  
L’objectif est d’obtenir des fichiers propres, cohérents et compatibles entre eux, afin de pouvoir les croiser dans la phase d’analyse.

Les jeux de données utilisés :
- **Demandes de valeurs foncières (DVF)** – Prix de l’immobilier
- **Effectifs étudiants (MESR)** – Localisation et importance des établissements d’enseignement supérieur
- **Indicateurs de loyers d’annonce (SDES)** – Niveaux du marché locatif privé
- **Population par âge (INSEE)** – Densité de population étudiante potentielle

Cette étape inclut :
1. La détection automatique des encodages et séparateurs.
2. Le nettoyage des noms de colonnes et formats.
3. La normalisation des identifiants de commune.
4. La préparation d’un fichier réduit et filtré pour la région Île-de-France.


## Importation des bibliothèques et paramètres généraux

Nous commençons par importer les bibliothèques nécessaires et définir les chemins des fichiers.  
Nous ajoutons également une configuration pour ignorer certains avertissements non critiques liés à `pandas`.


In [14]:
import os, io, csv, chardet, warnings
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore", message="Engine has switched to 'python'")

# Paramètres généraux
IDF_DEPS = {"75","77","78","91","92","93","94","95"}
DVF_PATH = "Demandes_de_valeurs_foncieres_geolocalisees.csv"
MESR_PATH = "Effectifs_etudiants_par_etablissements.csv"
LOYERS_PATH = "Indicateurs_de_loyers_annonce_par_commune_2.csv"
POP_PATH = "population_par_sexe_age.xlsx"
TMP_DVF_PATH = "dvf_idf_prixm2_tmp.csv"
CHUNK = 250_000


## Fonctions utilitaires

Plusieurs fonctions permettent d’automatiser la détection et le nettoyage :
- `sniff_encoding_and_sep()` : détecte automatiquement l’encodage et le séparateur d’un fichier CSV.
- `clean_cols()` : nettoie les noms de colonnes (minuscules, sans accents ni espaces).
- `ensure_code_insee()` : normalise les codes INSEE des communes à 5 chiffres.


In [15]:
def sniff_encoding_and_sep(path):
    """Détecte encodage + séparateur probable."""
    with open(path, "rb") as f:
        raw = f.read(1_000_000)
    for enc in ("utf-8-sig","utf-8","latin-1"):
        try:
            text = raw.decode(enc)
            break
        except UnicodeDecodeError:
            text = None
    if text is None:
        text = raw.decode("latin-1", errors="replace")
        enc = "latin-1"
    try:
        dialect = csv.Sniffer().sniff(text)
        sep = dialect.delimiter
    except Exception:
        counts = {",": text.count(","), ";": text.count(";"), "\t": text.count("\t")}
        sep = max(counts, key=counts.get)
    return enc, sep


def clean_cols(df):
    cols = (df.columns
              .str.strip()
              .str.replace(r"\s+", "_", regex=True)
              .str.replace(r"[^\w_]", "", regex=True)
              .str.lower())
    df.columns = cols
    return df


def ensure_code_insee(s):
    return s.astype(str).str.strip().str.replace(r"\.0$", "", regex=True).str.zfill(5)


## 1. Traitement du jeu DVF : prix de l'immobilier

Le jeu de données DVF est volumineux (plusieurs millions de lignes).  
Nous le lisons **par blocs (chunks)** afin de ne pas saturer la mémoire.  
Seules les ventes de logements situées en **Île-de-France** sont conservées.

Les colonnes utiles :
- `code_commune`, `code_departement`
- `valeur_fonciere`
- `surface_reelle_bati`
- `type_local`
- `nature_mutation`

Les valeurs aberrantes sont filtrées et le prix médian au m² est calculé par commune.


In [16]:
if os.path.exists(TMP_DVF_PATH):
    os.remove(TMP_DVF_PATH)

enc, sep = sniff_encoding_and_sep(DVF_PATH)
print(f"DVF -> enc='{enc}', sep='{repr(sep)}'")

dvf_usecols = [
    "nature_mutation","type_local","valeur_fonciere",
    "surface_reelle_bati","code_commune","code_departement"
]

first_write = True
rows_kept = 0
chunk_iter = pd.read_csv(
    DVF_PATH, sep=sep, encoding=enc, usecols=dvf_usecols,
    dtype={
        "nature_mutation":"category",
        "type_local":"category",
        "valeur_fonciere":"string",
        "surface_reelle_bati":"string",
        "code_commune":"string",
        "code_departement":"string",
    },
    chunksize=CHUNK, low_memory=True, on_bad_lines="skip"
)

for i,chunk in enumerate(chunk_iter, 1):
    chunk["code_departement"] = chunk["code_departement"].str.strip()
    chunk = chunk[chunk["code_departement"].isin(IDF_DEPS)]
    if chunk.empty: 
        continue
    chunk = chunk[
        chunk["type_local"].isin(["Appartement","Maison","APPARTEMENT","MAISON"])
        & (chunk["nature_mutation"] == "Vente")
    ]
    if chunk.empty:
        continue
    chunk["valeur_fonciere"] = pd.to_numeric(chunk["valeur_fonciere"].str.replace(",", "."), errors="coerce")
    chunk["surface_reelle_bati"] = pd.to_numeric(chunk["surface_reelle_bati"].str.replace(",", "."), errors="coerce")
    chunk = chunk.query("surface_reelle_bati >= 8 and surface_reelle_bati <= 300")
    if chunk.empty:
        continue
    chunk["code_commune"] = ensure_code_insee(chunk["code_commune"])
    chunk["prix_m2"] = chunk["valeur_fonciere"] / chunk["surface_reelle_bati"]
    out = chunk[["code_commune","prix_m2"]].dropna()
    if not out.empty:
        out.to_csv(TMP_DVF_PATH, mode="a", index=False, header=first_write)
        first_write = False
        rows_kept += len(out)
    if i % 10 == 0:
        print(f"… chunks traités: {i}, lignes conservées: {rows_kept:,}")

print(f"Fichier réduit DVF: {TMP_DVF_PATH} ({os.path.getsize(TMP_DVF_PATH)/1e6:.1f} Mo)")

dvf_commune = None
for j,mini in enumerate(pd.read_csv(TMP_DVF_PATH, chunksize=1_000_000), 1):
    g = mini.groupby("code_commune")["prix_m2"].median().to_frame("prix_median_m2")
    dvf_commune = g if dvf_commune is None else pd.concat([dvf_commune, g], axis=0)
dvf_commune = dvf_commune.groupby(dvf_commune.index).median().reset_index()
print("DVF agrégé (IDF) ->", dvf_commune.shape)
display(dvf_commune.head())


DVF -> enc='utf-8-sig', sep='',''
… chunks traités: 30, lignes conservées: 240,838
… chunks traités: 50, lignes conservées: 470,230
… chunks traités: 80, lignes conservées: 852,970
Fichier réduit DVF: dvf_idf_prixm2_tmp.csv (20.8 Mo)
DVF agrégé (IDF) -> (1286, 2)


,code_commune,prix_median_m2
0,75101,13750.000000
1,75102,12262.726488
2,75103,12884.615385
3,75104,13660.000000
4,75105,12882.307692


## 2. Traitement des autres jeux de données

### 2.1 Effectifs étudiants (MESR)
Lecture simple, nettoyage des colonnes et extraction automatique des coordonnées GPS.

### 2.2 Loyers (SDES)
Détection automatique de l’encodage via la librairie `chardet`.  
Nettoyage, renommage des colonnes et uniformisation du code commune.

### 2.3 Population (INSEE)
Lecture du fichier Excel après détection automatique de la ligne d’en-tête réelle.  
Nettoyage des noms de colonnes et conversion des pourcentages en nombres.


In [17]:
# MESR
effectifs_df = pd.read_csv(MESR_PATH, sep=";", encoding="utf-8-sig", low_memory=False, on_bad_lines="skip")
effectifs_df = clean_cols(effectifs_df)
if "gps" in effectifs_df.columns:
    gps = effectifs_df["gps"].astype(str).str.split(",", n=1, expand=True)
    effectifs_df["lat"] = pd.to_numeric(gps[0], errors="coerce")
    effectifs_df["lon"] = pd.to_numeric(gps[1], errors="coerce")
print("\nMESR ->", effectifs_df.shape)
display(effectifs_df.head(3))

# LOYERS
with open(LOYERS_PATH, 'rb') as f:
    raw = f.read(500_000)
enc_guess = chardet.detect(raw)
encoding_detected = enc_guess["encoding"] or "latin-1"
print(f"[INFO] Encodage détecté pour loyers : {encoding_detected}")

try:
    loyers_df = pd.read_csv(
        LOYERS_PATH, sep=";", encoding=encoding_detected, low_memory=False, on_bad_lines="skip"
    )
except UnicodeDecodeError:
    loyers_df = pd.read_csv(
        LOYERS_PATH, sep=";", encoding="latin-1", low_memory=False, on_bad_lines="skip"
    )
loyers_df = clean_cols(loyers_df)
for cand in ("insee_c","code_commune"):
    if cand in loyers_df.columns:
        loyers_df["code_commune"] = ensure_code_insee(loyers_df[cand])
        break
for cand in ("loypredm2","loy_median_m2","loyer_median_m2"):
    if cand in loyers_df.columns:
        loyers_df = loyers_df.rename(columns={cand: "loyer_median_m2"})
        break
print("\nLoyers ->", loyers_df.shape)
display(loyers_df.head(3))

# POPULATION
pop_raw = pd.read_excel(POP_PATH, header=None, engine="openpyxl")
header_row = pop_raw.apply(lambda r: r.astype(str).str.contains("Ensemble", case=False, na=False)).any(axis=1).idxmax()
population_df = pd.read_excel(POP_PATH, skiprows=header_row, engine="openpyxl")
population_df = clean_cols(population_df)
rename_map = {
    "ensemble": "population_totale",
    "part_des_femmes_en": "part_femmes_pct",
    "part_des_hommes_en": "part_hommes_pct",
    "part_des_0_a_24_ans_en": "part_0_24_pct",
    "part_des_25_a_59_ans_en": "part_25_59_pct",
    "part_des_60_ans_ou_plus_en": "part_60_plus_pct",
    "dont_part_des_75_ans_ou_plus_en": "part_75_plus_pct",
}
population_df = population_df.rename(columns={k:v for k,v in rename_map.items() if k in population_df.columns})
for col in [c for c in population_df.columns if c.endswith("_pct") or "population_totale" in c]:
    population_df[col] = pd.to_numeric(population_df[col], errors="coerce")
print("\nPopulation ->", population_df.shape)
display(population_df.head(5))



MESR -> (105660, 29)


,rentrée_universitaire,catégorie_détablissement,secteur_détablissement,code_uai_de_létablissement,sigle_de_létablissement,libellé_de_létablissement,libellé_complémentaire_de_létablissement,code_uai_de_la_composante,sigle_de_la_composante,libellé_de_la_composante,...,code_commune,commune,gps,degetu,degre_etudes,nombre_total_détudiants_inscrits_hors_doubles_inscriptions_universitécpge,dont_femmes,dont_hommes,lat,lon
0,2023,"Écoles de commerce, gestion et vente",Privé,0562044T,EC TECH SUP PR,GROUPE AFTEC,IPAC BACHELOR FACTORY VANNES,0562044T,EC TECH SUP PR,GROUPE AFTEC,...,56158,Plescop,"47.67764394859038, -2.80211043128016",3.0,BAC + 3,112,96,16,47.677644,-2.80211
1,2023,"Écoles de commerce, gestion et vente",Privé,0590346F,SKEMA LILLE,SKEMA BUSINESS SCHOOL LILLE,NaN,0060656F,E.COM,ECOLE DE COMMERCE,...,06152,Valbonne,"43.61328883276337, 7.056290282560445",6.0,BAC + 6 et plus,19,9,10,43.613289,7.05629
2,2023,"Écoles de commerce, gestion et vente",Privé,0590346F,SKEMA LILLE,SKEMA BUSINESS SCHOOL LILLE,NaN,216P0001,SKEMA CHINE,SKEMA BUSINESS SCHOOL,...,99216,Suzhou,NaN,4.0,BAC + 4,389,67,322,NaN,NaN


[INFO] Encodage détecté pour loyers : Windows-1252

Loyers -> (34970, 14)


,id_zone,insee_c,libgeo,epci,dep,reg,loyer_median_m2,lwripm2,upripm2,typpred,nbobs_com,nbobs_mail,r2_adj,code_commune
0,1,36077,Fontguenand,200040558,36,24,"9,18996654326032","7,19993079768473","11,7300412239243",maille,0,521,"0,608561344447099",36077
1,1,36092,Langé,200040558,36,24,"9,18996654326032","7,19993079768473","11,7300412239243",maille,0,521,"0,608561344447099",36092
2,1,36123,Mézières-en-Brenne,243600343,36,24,"9,18996654326032","7,19993079768473","11,7300412239243",maille,8,521,"0,608561344447099",36123



Population -> (104, 9)


,unnamed_0,unnamed_1,population_totale,part_des_femmes_en_,part_des_hommes_en_,part_des_0_à_24_ans_en_,part_des_25_à_59_ans_en_,part_des_60_ans_ou_plus_en_,dont_part_des_75_ans_ou_plus_en_
0,01,Ain,688626.0,50.7,49.3,29.7,44.8,25.5,9.3
1,02,Aisne,518817.0,51.1,48.9,28.9,41.7,29.4,10.6
2,03,Allier,332599.0,51.7,48.3,24.4,39.0,36.6,14.9
3,04,Alpes-de-Haute-Provence,169806.0,51.5,48.5,24.1,39.9,35.9,14.3
4,05,Hautes-Alpes,142006.0,51.1,48.9,23.1,41.1,35.8,14.0


# Partie 2 : Affichage et contrôle des dataset

Cette section permet de visualiser les résultats du nettoyage et de vérifier la cohérence des types de données.  
Nous affichons pour chaque jeu de données :
- le nombre de lignes et de colonnes ;
- les types de variables ;
- un aperçu des premières lignes.


In [18]:
for name, df in {
    "DVF_commune(IDF)": dvf_commune,
    "MESR": effectifs_df,
    "Loyers": loyers_df,
    "Population": population_df
}.items():
    print(f"\n=== {name} ===")
    print("Dimensions :", df.shape)
    print(df.dtypes.head())
    display(df.head(3))



=== DVF_commune(IDF) ===
Dimensions : (1286, 2)
code_commune        int64
prix_median_m2    float64
dtype: object


,code_commune,prix_median_m2
0,75101,13750.000000
1,75102,12262.726488
2,75103,12884.615385



=== MESR ===
Dimensions : (105660, 29)
rentrée_universitaire          int64
catégorie_détablissement      object
secteur_détablissement        object
code_uai_de_létablissement    object
sigle_de_létablissement       object
dtype: object


,rentrée_universitaire,catégorie_détablissement,secteur_détablissement,code_uai_de_létablissement,sigle_de_létablissement,libellé_de_létablissement,libellé_complémentaire_de_létablissement,code_uai_de_la_composante,sigle_de_la_composante,libellé_de_la_composante,...,code_commune,commune,gps,degetu,degre_etudes,nombre_total_détudiants_inscrits_hors_doubles_inscriptions_universitécpge,dont_femmes,dont_hommes,lat,lon
0,2023,"Écoles de commerce, gestion et vente",Privé,0562044T,EC TECH SUP PR,GROUPE AFTEC,IPAC BACHELOR FACTORY VANNES,0562044T,EC TECH SUP PR,GROUPE AFTEC,...,56158,Plescop,"47.67764394859038, -2.80211043128016",3.0,BAC + 3,112,96,16,47.677644,-2.80211
1,2023,"Écoles de commerce, gestion et vente",Privé,0590346F,SKEMA LILLE,SKEMA BUSINESS SCHOOL LILLE,NaN,0060656F,E.COM,ECOLE DE COMMERCE,...,06152,Valbonne,"43.61328883276337, 7.056290282560445",6.0,BAC + 6 et plus,19,9,10,43.613289,7.05629
2,2023,"Écoles de commerce, gestion et vente",Privé,0590346F,SKEMA LILLE,SKEMA BUSINESS SCHOOL LILLE,NaN,216P0001,SKEMA CHINE,SKEMA BUSINESS SCHOOL,...,99216,Suzhou,NaN,4.0,BAC + 4,389,67,322,NaN,NaN



=== Loyers ===
Dimensions : (34970, 14)
id_zone    object
insee_c    object
libgeo     object
epci       object
dep        object
dtype: object


,id_zone,insee_c,libgeo,epci,dep,reg,loyer_median_m2,lwripm2,upripm2,typpred,nbobs_com,nbobs_mail,r2_adj,code_commune
0,1,36077,Fontguenand,200040558,36,24,"9,18996654326032","7,19993079768473","11,7300412239243",maille,0,521,"0,608561344447099",36077
1,1,36092,Langé,200040558,36,24,"9,18996654326032","7,19993079768473","11,7300412239243",maille,0,521,"0,608561344447099",36092
2,1,36123,Mézières-en-Brenne,243600343,36,24,"9,18996654326032","7,19993079768473","11,7300412239243",maille,8,521,"0,608561344447099",36123



=== Population ===
Dimensions : (104, 9)
unnamed_0               object
unnamed_1               object
population_totale      float64
part_des_femmes_en_    float64
part_des_hommes_en_    float64
dtype: object


,unnamed_0,unnamed_1,population_totale,part_des_femmes_en_,part_des_hommes_en_,part_des_0_à_24_ans_en_,part_des_25_à_59_ans_en_,part_des_60_ans_ou_plus_en_,dont_part_des_75_ans_ou_plus_en_
0,01,Ain,688626.0,50.7,49.3,29.7,44.8,25.5,9.3
1,02,Aisne,518817.0,51.1,48.9,28.9,41.7,29.4,10.6
2,03,Allier,332599.0,51.7,48.3,24.4,39.0,36.6,14.9


# Partie 3 : Nettoyage complémentaire et préparation finale

Dans cette section, nous effectuons un **nettoyage complémentaire** sur l’ensemble des jeux de données
afin d’obtenir une base parfaitement cohérente et exploitable pour les analyses à venir.

Les étapes principales sont les suivantes :

1. **Uniformisation des codes communes** :  
   Pour garantir une fusion correcte entre les jeux (DVF, Loyers, MESR), tous les codes INSEE sont convertis
   en chaînes de 5 caractères (`str.zfill(5)`).

2. **Conversion des colonnes numériques** :  
   Le fichier SDES contient les loyers sous forme de texte avec des virgules comme séparateur décimal.  
   Nous les convertissons en valeurs flottantes pour permettre des comparaisons et calculs.

3. **Filtrage géographique des établissements MESR** :  
   Nous ne conservons que les établissements situés dans les départements de l’Île-de-France
   (codes 75, 77, 78, 91, 92, 93, 94, 95).

4. **Nettoyage des valeurs extrêmes ou aberrantes** :  
   - Suppression des prix fonciers trop faibles (inférieurs à 1000 €/m²)  
   - Suppression des loyers anormaux (inférieurs à 2 €/m²)

5. **Suppression des doublons et renommage** :  
   Les doublons sont supprimés et les colonnes clés sont renommées
   pour harmoniser les structures (`prix_m2_median`, `loyer_m2_median`).

6. **Contrôle final** :  
   Nous affichons pour chaque jeu de données nettoyé :
   - Les types de colonnes  
   - Un aperçu des premières lignes  
   - Le nombre de lignes restantes après filtrage

Ces opérations assurent la cohérence et la comparabilité entre les différentes sources
avant les étapes de fusion et d’analyse spatiale.


In [19]:


print("=== Nettoyage complémentaire en cours... ===")

# --- 1) Uniformisation des codes communes ---
for df_name, df in [("DVF", dvf_commune), ("Loyers", loyers_df), ("MESR", effectifs_df)]:
    if "code_commune" in df.columns:
        df["code_commune"] = (
            df["code_commune"]
            .astype(str)
            .str.strip()
            .str.replace(r"\.0$", "", regex=True)
            .str.zfill(5)
        )
        print(f"{df_name} : code_commune normalisé")

# --- 2) Conversion des loyers en valeurs numériques ---
if "loyer_median_m2" in loyers_df.columns:
    loyers_df["loyer_median_m2"] = (
        loyers_df["loyer_median_m2"]
        .astype(str)
        .str.replace(",", ".")
        .astype(float)
    )
    print("Loyers : conversion des valeurs de loyer_median_m2 en float")
elif "loyer_m2_median" in loyers_df.columns:
    loyers_df["loyer_m2_median"] = (
        loyers_df["loyer_m2_median"]
        .astype(str)
        .str.replace(",", ".")
        .astype(float)
    )
    print("Loyers : conversion des valeurs de loyer_m2_median en float")

# --- 3) Filtrage des établissements MESR en Île-de-France ---
idf_codes = ["75", "77", "78", "91", "92", "93", "94", "95"]
mesr_idf = effectifs_df[
    effectifs_df["code_commune"].astype(str).str[:2].isin(idf_codes)
].copy()
print(f"MESR : filtrage Île-de-France ({len(mesr_idf)} établissements)")

# --- 4) Filtrage des valeurs aberrantes ---
dvf_commune = dvf_commune.dropna(subset=["prix_median_m2"])
dvf_commune = dvf_commune[dvf_commune["prix_median_m2"] > 1000]

# Vérifie le nom exact de la colonne de loyers avant de filtrer
loyer_col = "loyer_median_m2" if "loyer_median_m2" in loyers_df.columns else "loyer_m2_median"
loyers_df = loyers_df.dropna(subset=[loyer_col])
loyers_df = loyers_df[loyers_df[loyer_col] > 2]

print(f"DVF : {len(dvf_commune)} communes après filtrage prix minimum")
print(f"Loyers : {len(loyers_df)} communes après filtrage loyer minimum")

# --- 5) Suppression des doublons ---
dvf_commune = dvf_commune.drop_duplicates(subset="code_commune")
loyers_df = loyers_df.drop_duplicates(subset="code_commune")
mesr_idf = mesr_idf.drop_duplicates(subset="code_commune")

# --- 6) Renommage cohérent des colonnes ---
dvf_commune = dvf_commune.rename(columns={"prix_median_m2": "prix_median_m2"})
if "loyer_median_m2" in loyers_df.columns:
    loyers_df = loyers_df.rename(columns={"loyer_median_m2": "loyer_m2_median"})

# --- 7) Vérification des types ---
print("\n--- Vérification rapide des types ---")
for name, df in {
    "DVF": dvf_commune,
    "Loyers": loyers_df,
    "MESR_IDF": mesr_idf,
}.items():
    print(f"\n{name}")
    print(df.dtypes.head())

# --- 8) Aperçu final des données nettoyées ---
print("\nAperçu DVF :")
display(dvf_commune.head(3))

print("\nAperçu Loyers :")
display(loyers_df.head(3))

print("\nAperçu MESR (Île-de-France) :")
display(mesr_idf.head(3))

print("\n=== Nettoyage final terminé avec succès ===")


=== Nettoyage complémentaire en cours... ===
DVF : code_commune normalisé
Loyers : code_commune normalisé
MESR : code_commune normalisé
Loyers : conversion des valeurs de loyer_median_m2 en float
MESR : filtrage Île-de-France (21608 établissements)
DVF : 1283 communes après filtrage prix minimum
Loyers : 34970 communes après filtrage loyer minimum

--- Vérification rapide des types ---

DVF
code_commune       object
prix_median_m2    float64
dtype: object

Loyers
id_zone    object
insee_c    object
libgeo     object
epci       object
dep        object
dtype: object

MESR_IDF
rentrée_universitaire          int64
catégorie_détablissement      object
secteur_détablissement        object
code_uai_de_létablissement    object
sigle_de_létablissement       object
dtype: object

Aperçu DVF :


,code_commune,prix_median_m2
0,75101,13750.000000
1,75102,12262.726488
2,75103,12884.615385



Aperçu Loyers :


,id_zone,insee_c,libgeo,epci,dep,reg,loyer_m2_median,lwripm2,upripm2,typpred,nbobs_com,nbobs_mail,r2_adj,code_commune
0,1,36077,Fontguenand,200040558,36,24,9.189967,"7,19993079768473","11,7300412239243",maille,0,521,"0,608561344447099",36077
1,1,36092,Langé,200040558,36,24,9.189967,"7,19993079768473","11,7300412239243",maille,0,521,"0,608561344447099",36092
2,1,36123,Mézières-en-Brenne,243600343,36,24,9.189967,"7,19993079768473","11,7300412239243",maille,8,521,"0,608561344447099",36123



Aperçu MESR (Île-de-France) :


,rentrée_universitaire,catégorie_détablissement,secteur_détablissement,code_uai_de_létablissement,sigle_de_létablissement,libellé_de_létablissement,libellé_complémentaire_de_létablissement,code_uai_de_la_composante,sigle_de_la_composante,libellé_de_la_composante,...,code_commune,commune,gps,degetu,degre_etudes,nombre_total_détudiants_inscrits_hors_doubles_inscriptions_universitécpge,dont_femmes,dont_hommes,lat,lon
3,2023,"Écoles de commerce, gestion et vente",Privé,0590350K,EDHEC LILLE,EDHEC BUSINESS SCHOOL,MEMBRE FUPL,0755719J,EDHEC,CENTRE DE FORMATION CONTINUE,...,75102,Paris 2e,NaN,5.0,BAC + 5,387,162,225,NaN,NaN
5,2023,"Écoles de commerce, gestion et vente",Privé,0593202K,IESEG LILLE,INST ECO SCIENT GESTION LILLE,MEMBRE FUPL,0922663V,IESEG BOULOGNE,INST ECO SC GEST BOULOGNE-BILL,...,92050,Nanterre,"48.89739112970496, 2.224693366221823",4.0,BAC + 4,1022,535,487,48.897391,2.224693
11,2023,"Écoles de commerce, gestion et vente",Privé,0751698N,PIGIER,ETAB ENSEIGNT SUPERIEUR PRIVE,LA COMPAGNIE DE FORMATION PIGI,0751698N,PIGIER,ETAB ENSEIGNT SUPERIEUR PRIVE,...,75119,Paris 19e,"48.896147655260954, 2.380225384944163",1.0,BAC + 1,212,133,79,48.896148,2.380225



=== Nettoyage final terminé avec succès ===


# Partie 4 – Synthèse du nettoyage et préparation à la fusion

## Bilan général du travail effectué

Dans cette première phase du projet, nous avons procédé à un travail complet de **chargement, exploration et nettoyage** des quatre jeux de données nécessaires à l’analyse du marché du logement étudiant en Île-de-France.

Les étapes principales ont été :
1. **Chargement des fichiers bruts** (formats CSV et Excel) depuis les sources officielles (data.gouv.fr, INSEE, MESR).
2. **Correction des encodages et séparateurs** afin d’assurer une lecture homogène des données malgré des structures différentes.
3. **Nettoyage approfondi** :  
   - suppression des lignes ou colonnes inutiles,  
   - normalisation des noms de colonnes,  
   - conversion des types de données (`float`, `int`, `str`),  
   - harmonisation des identifiants communaux (`code_commune`).
4. **Filtrage des valeurs aberrantes** pour retirer les montants incohérents (prix foncier trop bas, loyers trop faibles, etc.).
5. **Restriction géographique** sur l’Île-de-France pour le dataset MESR, en ne conservant que les établissements situés dans cette région.


## État final des datasets

Les trois principaux jeux de données nécessaires à la fusion sont désormais **normalisés et homogènes** :

| Dataset | Description | Nombre d’enregistrements |
|----------|--------------|---------------------------|
| **DVF** | Prix du foncier par commune (€/m²) | 1 283 |
| **Loyers** | Loyer médian du marché privé (€/m²/mois) | 34 970 |
| **MESR (IDF)** | Établissements d’enseignement supérieur en Île-de-France | 21 608 |

Ces trois tables partagent un champ clé commun : `code_commune`,  
ce qui permettra une **fusion directe** dans la prochaine étape.

Le quatrième jeu de données, **INSEE – Population par âge**, n’a pas encore été exploité ici.  
Il sera intégré ultérieurement afin de calculer des indicateurs de densité étudiante par commune.


## Difficultés rencontrées et solutions appliquées

- **Problèmes d’encodage** (`utf-8`, `latin-1`) : résolus par une détection automatique et un décodage robuste.  
- **Colonnes mal formatées** (présence d’espaces, accents, ou caractères spéciaux) : corrigées via une fonction de nettoyage systématique.  
- **Volume de données important dans DVF** : traitement en **chunks** pour éviter les erreurs mémoire et conserver uniquement les logements de surface réaliste.  
- **Séparateurs incohérents** dans certains fichiers (`;`, `,`) : test automatique des deux formats avant chargement.

Ces ajustements ont permis d’obtenir des jeux de données cohérents, exploitables et compatibles entre eux pour la suite du projet.


## Étape suivante

Le prochain notebook (**02_indicateurs_crous.ipynb**) utilisera ces données nettoyées pour :
- fusionner les informations foncières, locatives et universitaires,  
- créer de nouveaux indicateurs pertinents pour le CROUS (ex : ratio loyer/prix foncier, attractivité communale, densité étudiante),  
- et générer une base unique de travail pour l’analyse et la visualisation.

Avant cela, nous exportons ci-dessous les fichiers nettoyés afin de pouvoir les réutiliser facilement.


In [20]:
# Export des données nettoyées

# Export des trois jeux principaux
dvf_commune.to_csv("dvf_commune_clean.csv", index=False, encoding="utf-8-sig")
loyers_df.to_csv("loyers_clean.csv", index=False, encoding="utf-8-sig")
mesr_idf.to_csv("mesr_idf_clean.csv", index=False, encoding="utf-8-sig")

print("=== Export terminé avec succès ===")
print("Fichiers créés :")
print("- dvf_commune_clean.csv  → Données foncières nettoyées")
print("- loyers_clean.csv       → Données de loyers médians")
print("- mesr_idf_clean.csv     → Établissements MESR en Île-de-France")



=== Export terminé avec succès ===
Fichiers créés :
- dvf_commune_clean.csv  → Données foncières nettoyées
- loyers_clean.csv       → Données de loyers médians
- mesr_idf_clean.csv     → Établissements MESR en Île-de-France


# Clôture du Notebook 1

Le travail de préparation est désormais terminé.  
Les données sont prêtes pour la phase suivante : **fusion et construction d’indicateurs**.  
Le notebook suivant se concentrera sur la mise en relation des différentes sources et la création de variables utiles à la décision du CROUS de l’Île-de-France.
